# Homework 5

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

########## Graph-Search-Related Utilities and Class Definitions ##########

from collections import namedtuple, defaultdict
from typing import Optional, List
import heapq as hq
import numpy as np
import math
import random


def grid_successor_fn(state):
  """Helper for testing heuristic search.
  """
  arrival_costs = np.array([
    [1, 1, 8, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 8, 1, 1, 1],
    [1, 1, 1, 8, 1],
    [1, 1, 2, 1, 1],
  ], dtype=float)

  act_to_delta = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
  }

  r, c = state

  for act, (dr, dc) in act_to_delta.items():
    new_r, new_c = r + dr, c + dc
    # Check if in bounds
    if not (0 <= new_r < arrival_costs.shape[0] and \
            0 <= new_c < arrival_costs.shape[1]):
      continue
    # Valid action
    yield (act, (new_r, new_c), arrival_costs[new_r, new_c])


def grid_check_goal_fn(state):
  """Helper for testing heuristic search.
  """
  # Bottom right corner of grid
  return state == (4, 4)


def grid_heuristic_fn(state):
  """Helper for testing heuristic search.
  """
  # Manhattan distance
  return abs(state[0] - 4) + abs(state[1] - 4)


# A useful data structure for heuristic search
Node = namedtuple("Node", ["state", "parent", "action", "cost", "g"])


########## RRT-Related Utilities and Class Definitions ##########

# Useful geometric data structures and methods.
def euclidean_distance(a, b):
  return math.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2)


RobotConfig = namedtuple('RobotConfig', ['x', 'y'])


class Circle(namedtuple('Circle', ['x', 'y', 'r'])):
  def collide(self, config: RobotConfig):
    return euclidean_distance(config, self) < self.r


class RobotPlanningProblem:
  """
  A robot planning problem definition contains the configuration space of the robot
  and all obstacles represented as geometries in the configuration space.

  In this problem set, we will consider a simple 2D map, where the robot configuration
  space is represented as a tuple (x, y).

  In the __init__ definition, we will define the range of x and y values of the robot's configuration,
  as well as a list of obstacles. See the code for details.
  """
  def __init__(self):
    # The ma
    self.x_range = (-10, 10)  # range of the x value.
    self.y_range = (-10, 10)  # range of the y value.
    self.obstacles = [
      Circle(0, 0, 4),
      Circle(-5, -5, 2),
      Circle(3, -4, 3),
      Circle(-3, 4, 1)
    ]

  def sample(self):
    """Generate a random sample in the robot configuration space."""
    return RobotConfig(random.uniform(*self.x_range), random.uniform(*self.y_range))

  def distance(self, a: RobotConfig, b: RobotConfig):
    """Return the distance between two robot configurations."""
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

  def collide(self, x: RobotConfig):
    """Test if the input robot configuration collides with some obstacles."""
    for obs in self.obstacles:
      if obs.collide(x):
        return True
    return False


class RRTNode:
  """Node of a rapidly-exploring random tree.

  Each node is associated with a robot configuration.
  It also keeps track of
    - the parent in the tree.
    - a list of children nodes in the tree.

  If the node itself is the node for an RRT, node.parent will be set to None.
  If the node is a leaf node in an RRT, node.children will be an empty list.
  """

  config: RobotConfig # Robot state
  parent: Optional['RRTNode']  # parent node
  children: List['RRTNode']  # children

  def __init__(self, config: RobotConfig, parent: Optional['RRTNode'] = None):
    self.config = config
    self.parent = parent
    self.children = list()

  def add_child(self, other):
    """
    Register another node as the child of the current node.

    Args:
      other (RRTNode): the other RRTNode.

    Returns:
      self (RRTNode)
    """
    self.children.append(other)
    return self

  def attach_to(self, parent):
    """Attach the current node to another node. This function will:
      - set the parent of self to parent.
      - add self to parent.children.

    Args:
      parent (RRTNode): the parent node.

    Returns:
      self (RRTNode)
    """
    self.parent = parent
    self.parent.add_child(self)
    return self

  def backtrace(self, config=True):
    """Return the path from root to the current node. For example, if the tree is:

    s0 --> s1 -> s2 -> S3 -> s4
       \
        -> s5 -> s6

    Calling s6.backtrace() will return the list: [s0, s5, s6].

    Args:
      config (bool): if True, return a list of robot configurations.
      Otherwise, return the RRTNode list.

    Returns:
      path (List[RobotConfig]): the path from root to the current node.
    """
    path = list()

    def dfs(x):
      nonlocal path
      if x.parent is not None:
        dfs(x.parent)
      path.append(x.config if config else x)

    try:
      dfs(self)
      return path
    finally:
      del dfs

  def __repr__(self):
      return f'RRTNode(config={self.config}, parent={self.parent})'


class RRT:
  """The RRT tree.

  The constructor of RRT takes two parameters:
    problem (RobotPlanningProblem): an instance of the robot planning problem.
      It will define the distance metric in the configuration space.
    roots (List[RRTNode]): a list of RRTNode instances. They are the "roots" of the RRT.
      Note that it is a list of nodes. So if you want to construct an RRT with a single
      root node, please use RRT(problem, [your_single_node]).

  Note that the RRT class does not explicitly manages all nodes in the tree.
  It only keeps track of the root nodes of the tree.
  To get all nodes in an RRT tree, we need to do a tree travesal from the roots.
  We have provided a helper function traverse_rrt_bfs for you to do that.

  There are two useful helper functions in RRT class.
    - extend: which attaches a new node to an existing node in the tree.
    - nearest: which finds the node in the RRT that is most close to the query node.
  See the function docstrings for details.
  """

  def __init__(self, problem: RobotPlanningProblem, roots: List[RRTNode]):
    if isinstance(roots, RRTNode):
      roots = [roots]
    else:
      assert isinstance(roots, list)

    self.problem = problem
    self.roots = roots
    self.size = len(roots)

  def extend(self, parent: RRTNode, child_config: RobotConfig) -> RobotConfig:
    """Create a new RRTNode from a robot configuration and set its parent.

    Basically, it creates a new RRTNode instance from child_config and attach this
    new node to the `parent` node. We recommend you to use this function
    instead of doing it yourself because it will help you keep track of the size
    of the RRT, which may be useful for your debugging purpose.

    Args:
      parent (RRTNode): the parent node (in the current RRT).
      child_config (RobotConig): the robot configuration of the new node.

    Returns:
      child (RRTNode): the new node.
    """
    child = RRTNode(child_config).attach_to(parent)
    self.size += 1
    return child

  def nearest(self, config: RobotConfig) -> RRTNode:
    """Return the RRTNode in the tree that is closest to the target configuration.
    We strongly suggest you to take a look at the function definition to see
    the example usage of function `traverse_rrt_bfs` and `problem.distance`.

    Args:
      config (RobotConfig): the query robot config.

    Returns:
      best_node (RRTNode): a node in the RRT that is closest to the query configuration
        based on the distance metric in the configuration space.
    """
    best_node, best_value = None, np.inf
    for node in traverse_rrt_bfs(self.roots):
      distance = self.problem.distance(node.config, config)
      if distance < best_value:
        best_value = distance
        best_node = node

    return best_node


def traverse_rrt_bfs(nodes: List[RRTNode]) -> List[RRTNode]:
  """Run breath-first search to return all RRTNodes that are descendants of
  the input list of RRTNodes.

  Args:
    nodes: a list of RRTNodes as the seed nodes.

  Returns:
    a list of RRTNodes that are descendants of the input lists (including the input list).
  """
  queue = nodes.copy()
  results = list()

  while len(queue) > 0:
    x = queue[0]
    queue = queue[1:]
    results.append(x)
    for y in x.children:
      queue.append(y)

  return results


########## Test-Related Tools for RRT ##########


def check_path_smooth(problem, path, max_diff, eps=1e-5):
  """Check if a path is collision-free and satisfies the max_diff constraint."""
  for config1, config2 in zip(path[:-1], path[1:]):
    if any(abs(config1[i] - config2[i]) > max_diff + eps for i in range(len(config1))):
      return False
  for config in path:
    if problem.collide(config):
      return False
  return True


def check_path_smooth_and_successful(problem, path, initial_config, goal_config, max_diff, eps=1e-5):
  """Check if a path is collision-free, satisfies the max_diff constraint, and connects initial_config and goal_config."""
  return np.allclose(path[0], initial_config, atol=eps) and np.allclose(path[-1], goal_config, atol=eps) and check_path_smooth(problem, path, max_diff, eps=eps)


########## Visualization Tools for RRT ##########


def visualize_map(problem, canvas_size=(512, 512)) -> 'Image':
  """Visualize a 2D environment.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    canvas_size (Tuple[int]): width and height of the canvas.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = Image.new('RGB', canvas_size, color='white')
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for obs in problem.obstacles:
    if isinstance(obs, Circle):
      AABB = (*gen_xy(obs.x - obs.r, obs.y + obs.r), *gen_xy(obs.x + obs.r, obs.y - obs.r))
      draw.ellipse(AABB, fill='gray', outline='gray')

  return image


def visualize_path(problem: RobotPlanningProblem, path: List[RobotConfig], canvas_size=(512, 512), show_nodes=True) -> 'Image':
  """Visualize a path in a 2D environment. The starting point will be marked as blue. The end point will be marked as green.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    path (List[RobotConfig]): a list of RobotConfig.
    canvas_size (Tuple[int]): width and height of the canvas.
    show_nodes (bool): if True, the function will visualize all nodes along the path.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = visualize_map(problem, canvas_size=canvas_size)
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for config1, config2 in zip(path[:-1], path[1:]):
    draw.line((*gen_xy(config1.x, config1.y), *gen_xy(config2.x, config2.y)), fill='red')

  if show_nodes:
    for config in path[1:-1]:
      draw.rectangle([
        *gen_xy(config.x - 0.1, config.y - 0.1),
        *gen_xy(config.x + 0.1, config.y + 0.1)
      ], fill='red')    

  draw.rectangle([
    *gen_xy(path[0].x - 0.3, path[0].y - 0.3),
    *gen_xy(path[0].x + 0.3, path[0].y + 0.3)
  ], fill='blue')
  draw.rectangle([
    *gen_xy(path[-1].x - 0.3, path[-1].y - 0.3),
    *gen_xy(path[-1].x + 0.3, path[-1].y + 0.3)
  ], fill='green')

  return image


def visualize_rrts(problem: RobotPlanningProblem, rrts: List[RRT], initial_config: RobotConfig, goal_config: RobotConfig, canvas_size=(512, 512), show_nodes=True) -> 'Image':
  """Visualize RRTs in a 2D environment. The initial config will be marked as blue. The goal config will be marked as green.

  Args:
    problem (RobotPlanningProblem): the robot planning problem.
    rrts (List[RRT]): a list of RRTs. Currently the function supports up to two rrts. The first one will show in red,
      while the second one wil show in blue.
    initial_config (RobotConfig): the initial config of the planning task.
    goal_config (RobotConfig): the goal config of the planning task.
    canvas_size (Tuple[int]): width and height of the canvas.
    show_nodes (bool): if True, the function will visualize all nodes in the tree.

  Returns:
    a PIL image of the visualization.
  """
  from PIL import Image, ImageDraw

  image = visualize_map(problem, canvas_size=canvas_size)
  draw = ImageDraw.Draw(image)

  def gen_xy(x, y):
    return (
      int((x - problem.x_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[0]),
      canvas_size[1] - int((y - problem.y_range[0]) / (problem.x_range[1] - problem.x_range[0]) * canvas_size[1])
    )

  for rrt, color in zip(rrts, ['red', 'blue']):
    for node in traverse_rrt_bfs(rrt.roots):
      if node.parent is not None:
        draw.line((*gen_xy(node.parent.config.x, node.parent.config.y), *gen_xy(node.config.x, node.config.y)), fill=color)

      if show_nodes:
        draw.rectangle([
          *gen_xy(node.config.x - 0.1, node.config.y - 0.1),
          *gen_xy(node.config.x + 0.1, node.config.y + 0.1)
        ], fill='red')

  draw.rectangle([
    *gen_xy(initial_config.x - 0.3, initial_config.y - 0.3),
    *gen_xy(initial_config.x + 0.3, initial_config.y + 0.3)
  ], fill='blue')
  draw.rectangle([
    *gen_xy(goal_config.x - 0.3, goal_config.y - 0.3),
    *gen_xy(goal_config.x + 0.3, goal_config.y + 0.3)
  ], fill='green')

  return image




## Problems

### Heuristic Search
Complete an implementation of heuristic search, encompassing A*, GBFS, or UCS.  You can assume any heuristics are consistent.

For reference, our solution is **45** lines of code.

In [ ]:
def run_heuristic_search(initial_state, check_goal, get_successors, get_priority, max_expansions=1000):
  """A generic heuristic search implementation.

  Depending on get_priority, can implement A*, GBFS, or UCS.

  Important: for determinism (and to make sure our tests pass),
  please break ties using the state itself. For example,
  if you would've otherwise sorted by get_priority(node), you
  should now sort by (get_priority(node), node.state).

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    get_priority: A callable that takes a Node and returns a
        float priority, with lower better, for the priority
        queue. This function is what switches between different
        versions of heurstic search.
    max_expansions: An int maximum number of nodes to expand
        before giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# We will test this implementation more thoroughly with the
# specific heuristic search algorithms that follow
initial_state = (0, 0)
get_priority_fn = lambda node: 0
result = run_heuristic_search(initial_state, grid_check_goal_fn,
    grid_successor_fn, get_priority_fn)
assert len(result) == 4

print('Tests passed.')

### Uniform Cost Search
Use your implementation of `run_heuristic_search` to implement uniform cost search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_uniform_cost_search(initial_state, check_goal, get_successors, max_expansions=1000):
  """Uniform-cost search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_uniform_cost_search(
    initial_state, grid_check_goal_fn, grid_successor_fn)
assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
assert num_expansions == 22

print('Tests passed.')

### A* Search
Use your implementation of `run_heuristic_search` to implement A* search.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_astar_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """A* search.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_astar_search(
    initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
assert state_sequence == [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (4, 2), (4, 3), (4, 4)]
assert action_sequence == ['down', 'down', 'down', 'right', 'right', 'down', 'right', 'right']
assert cost_sequence == [1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]
assert num_expansions == 11

print('Tests passed.')

### Greedy Best-First Search
Use your implementation of `run_heuristic_search` to implement GBFS.

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `run_heuristic_search`. You may not need to use all of them.

In [ ]:
def run_greedy_best_first_search(initial_state, check_goal, get_successors, heuristic, max_expansions=1000):
  """GBFS.

  Use your implementation of `run_heuristic_search`.

  Args:
    initial_state: A hashable representation of state.
    check_goal: A callable that takes a state and returns True
        if the state is a goal.
    get_successors: A callable that takes a state and returns an
        iterable of (action, next state, cost).
    heuristic: A callable that takes a state and returns an
        estimated cost-to-go (must be nonnegative).
    max_expansions: An int maximum number of nodes to expand
        because giving up.

  Returns:
    state_sequence: A list of states.
    action_sequence: A list of actions.
    cost_sequence: A list of costs.
    num_node_expansions: An int.

  Raises:
    error: ValueError, if no plan was found.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
# If your results do not match the expected ones, make sure that you are tiebreaking
# as described in the docstring for `run_heuristic_search`.
initial_state = (0, 0)
state_sequence, action_sequence, cost_sequence, num_expansions = run_greedy_best_first_search(
    initial_state, grid_check_goal_fn, grid_successor_fn, grid_heuristic_fn)
assert state_sequence == [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
assert action_sequence == ['right', 'right', 'right', 'right', 'down', 'down', 'down', 'down']
assert cost_sequence == [1.0, 8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
assert num_expansions == 8

print('Tests passed.')